# NewImage建表
* 

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
from impala.dbapi import connect
from impala.util import as_pandas
import json
import pymysql
import datetime
import random
from tqdm.auto import tqdm
tqdm.pandas(desc="Processing!")

## 游戏

In [7]:
def get_game_id():
    """ 中间件关联游戏ID """
    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [9]:
game = get_game_id()

## 建表

In [1]:
from impala.dbapi import connect
from impala.util import as_pandas

In [55]:
df = df[['image_id','image_name','label_ids','game_ids']]
df.drop_duplicates(inplace=True)  # keep=first

In [56]:
df['score'] = np.nan

In [57]:
df['label_ids'] = df['label_ids'].astype(str)
df['label_ids'] = df['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
# df['label_ids'] = pd.to_numeric(df['label_ids'], errors='coerce')

In [58]:
df.head()

,image_id,image_name,label_ids,game_ids,score
0,23237,SSR1399-横版（混剪）,19,1000840;1000993;1001258;1001155;1001049,NaN
1,23509,SSR1414-竖版（3D混剪）,20,1001379;1001400;1000993;1000840;1001049;100125...,NaN
3,23552,SSR1419-横版（3D混剪）,20,1000840;1000993;1001379;1001155;1001049;100125...,NaN
4,23846,SSR1484-横版（3D混剪）,20,1000993;1001379;1000840;1001155;1001400;1001049,NaN
5,23886,SSR1498-横版（3D混剪）,20,1000993;1001379;1001155;1000840;1001402,NaN


In [59]:
df['game_ids'].iloc[0]

'1000840;1000993;1001258;1001155;1001049'

In [65]:
df.to_csv('./new_image.csv', index=0, encoding='utf_8_sig',header=None)

In [6]:
def create_new_images():
    """ image新素材建表 """
    conn = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = '''
        CREATE TABLE test.new_image_test (
        image_id int,
        image_name varchar(255), 
        label_ids varchar(255),
        game_ids varchar(255),
        score int,
        label int,
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LINES TERMINATED BY '\n'
    '''
    cursor.execute(sql_engine)
    cursor.execute(sql)

    # 关闭链接
    cursor.close()
    conn.close()
    
    print("[INFO]：建表成功！")
    
    
def insert_new_images(data):
    """ image新素材插入 """
    conn = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = '''
        
    '''
    cursor.execute(sql_engine)
    cursor.execute(sql)

    # 关闭链接
    cursor.close()
    conn.close()
    
    print("[INFO]：插入成功！")
    
    
def get_new_images():
    """ image新素材提取 """
    conn = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,image_name,label_ids,game_ids from test.new_image_test where score is null'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = pd.to_numeric(result['label_ids'], errors='coerce')

    # 关闭链接
    cursor.close()
    conn.close()
    
    print("[INFO]：提取成功！")
    
    return result

In [7]:
create_new_images()

HiveServer2Error: Error while compiling statement: FAILED: ParseException line 9:8 cannot recognize input near ')' 'ROW' 'FORMAT' in column name or primary key or foreign key

In [72]:
# insert_new_images(df)

In [131]:
# get_new_images()

## 更新

In [ ]:
# os.system("hadoop fs -rm -r hdfs://masters/user/hive/warehouse/test.db/new_image_test)
# os.system("hadoop fs -mkdir hdfs://masters/user/hive/warehouse/test.db/new_image_test)
# os.system("hadoop fs -put new_image.csv hdfs://masters/user/hive/warehouse/test.db/new_image_test)
# os.system("hive -e \"msck repair table test.new_image_test\"")

In [ ]:
# =============================

In [124]:
def get_new_images(days):
    """ 实时更新create_time近3天内，没有评分的素材进入测新库test.new_image_test """
    
    # 游标1
    conn1 = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cur1 = conn1.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,label_ids,score from dws.dws_image_score_d where media_id=10 and dt>=\'2021-01-01\''
    cur1.execute(sql_engine)
    cur1.execute(sql)
    score_part = as_pandas(cur1)
    
    # 游标2
    conn2 = pymysql.connect(host='192.168.0.65', port=3306, user='readonly',
                               passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur2 = conn2.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
            /*手动查询*/ 
            SELECT
                a.image_id,
                a.image_name,
                a.label_ids,
                a.game_ids
            FROM
                db_ptom.ptom_image_info a
            WHERE
                a.create_time >= date( NOW() - INTERVAL 72 HOUR )
        '''
    cur2.execute(sql)
    image_part = pd.read_sql(sql, conn2)

    # 关闭链接
    cur1.close()
    conn1.close()
    cur2.close()
    conn2.close()
    
    return score_part, image_part

In [125]:
score_df, image_df = get_new_images(3)

In [127]:
# 匹配
def _filter(x, data):
    return True if not any(data['image_id'] == x.image_id) else False
image_df = image_df[image_df.progress_apply(lambda x:_filter(x, score_df), axis=1)]  # 近3天 - 没有评分记录的素材 - 新素材

Processing!:   0%|          | 0/178 [00:00<?, ?it/s]

In [132]:
image_df

,image_id,image_name,label_ids,game_ids
0,34680,SSR3463-竖版（3D混剪）,20,"1001703,1001687,1001661,1001649,1001643,100163..."
1,34681,GZJPS1080=A-ZH+ZJJ+LJW英雄救美(成功版）,,"1000741,1000744,1000747,1000748,1000768,100076..."
2,34682,GZJPS1083=B-YH失败选项,,"1000741,1000744,1000747,1000748,1000768,100076..."
3,34683,QMZT214-zrj,None,"1001465,1001466,1001473,1001493,1001545,100160..."
4,34684,QMZT215-zrj,None,"1001465,1001466,1001473,1001493,1001545,100160..."
...,...,...,...,...
173,34876,QMZZ1154-横版ZDC（3D混剪）（TT）,124,"1001713,1001702,1001700,1001693,1001692,100168..."
174,34877,QMZZ1155-竖版ZDC(帝国原生)(TT)(CS),"124,156","1001713,1001702,1001700,1001693,1001692,100168..."
175,34878,QMZZ1157-横版LL（TT）,"114,156","1001713,1001702,1001700,1001693,1001692,100168..."
176,34879,GZJPS1091=B-ZH+ZJJ+XHD牛夫人,,"1000741,1000744,1000747,1000748,1000768,100076..."


## 组合

In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))
from image_score_pre_new import NewImageScore

In [2]:
def fenpei(score_image_group, ad_account_id_group, columns, capacity):
    """ [资源分配]

        Args:
            score_image_group ([dataframe]): [新素材集]
            ad_account_id_group ([list]): [账号集]
            capacity ([int]): [容积限制]
    """
    dist_image_ids = score_image_group['image_id'].value_counts().index
    plan = pd.DataFrame(columns=columns)
    
    for image_id in sorted(dist_image_ids, key=lambda x: random.random()):  ## 按异质id标签遍历
        # 素材下新建计划
        sample = random.choice(score_image_group[score_image_group['image_id'] == image_id][['label_ids', 'score']].values)
        label = sample[0]
        score = sample[1]
        plan_part = pd.DataFrame({'image_id': [image_id], 'label_ids': [label.split(',')], 'image_score': [score]})
        plan = plan.append(plan_part)
    # 复制3条
    plan = pd.DataFrame(np.repeat(plan.values, 3, axis=0), columns=columns)
    # plan['ad_account_id'] = np.nan
    
    # 分配账号
    ad_counts = np.array([0] * len(ad_account_id_group), dtype=int)
    ad_account_id_group = np.array(ad_account_id_group, dtype=int)
    
    for image_id in dist_image_ids:
        # index = ad_counts.argsort()[:3]
        # ad_asso = ad_account_id_group[index[ad_counts[index] < capacity]]
        ad_asso = ad_account_id_group[ad_counts.argsort()[:3]]
        plan['ad_account_id'].loc[plan['image_id'] == image_id] = ad_asso
        ad_counts[ad_counts.argsort()[:3]] += 1
    # 容积过滤
    ad_caps = ad_account_id_group[ad_counts > capacity]
    if not ad_caps:
        return plan.reset_index(drop=True)
    else:
        for ad_cap in ad_caps:
            num_del = ad_counts[ad_account_id_group == ad_cap] - capacity
            dat_del = plan.loc[plan['ad_account_id'] == ad_cap].sample(num_del)
            plan = pd.merge(plan, dat_del, how='left', indicator=True).query("_merge=='left_only'").drop('_merge', 1)
        return plan.reset_index(drop=True)

In [3]:
ad = [7794, 7852, 7853, 7854, 7855]
col = ['ad_account_id', 'image_id', 'label_ids','image_score']
cap = 10
image_score_pre = NewImageScore()
img = image_score_pre()

KeyError: "None of [Index(['label_ids_value', 'game_ids_value'], dtype='object')] are in the [columns]"

In [97]:
import random

In [103]:
a = fenpei(img, ad, col, cap)

## 观察末日新素材
* 新素材

In [4]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [5]:
from media_toutiao.script.realtime_df import get_game_id

In [12]:
true_game_ids = get_game_id(1056)

In [13]:
# 88数据库 游标1
conn1 = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
            password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
cur1 = conn1.cursor()
sql_engine = 'set hive.execution.engine=tez'
sql = 'select image_id,label_ids,score from dws.dws_image_score_d where media_id=10 and dt>=\'2021-01-01\''
cur1.execute(sql_engine)
cur1.execute(sql)
score_df = as_pandas(cur1)

# 65数据库 游标2
conn2 = pymysql.connect(host='192.168.0.65', port=3306, user='readonly',
                        passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
cur2 = conn2.cursor(cursor=pymysql.cursors.DictCursor)
sql = '''
        /*手动查询*/ 
        SELECT
            a.image_id,
            a.image_name,
            a.label_ids,
            a.game_ids
        FROM
            db_ptom.ptom_image_info a
        WHERE
            a.create_time >= date(NOW() - INTERVAL 72 HOUR)  ## 最近3天
    '''
cur2.execute(sql)
image_df = pd.read_sql(sql, conn2)
image_df['label_ids'] = image_df['label_ids'].str.replace( ',', ';')  ## 避免分隔符冲突
image_df['game_ids'] = image_df['game_ids'].str.replace( ',', ';')  ## 避免分隔符冲突
# 新素材的label_id对齐
image_df = image_df.loc[image_df.apply(lambda x:True if not pd.isnull(x['label_ids']) and str(20) in x['label_ids'] else False, axis=1)]
# 新素材的game_ids对齐
true_game_ids = [str(game_id['game_id']) for game_id in true_game_ids]
image_df = image_df.loc[image_df.apply(lambda x:True if not pd.isnull(x['game_ids']) and len(list(set(true_game_ids).intersection(set(x['game_ids'].split(';'))))) > 0 else False, axis=1)]

# 关闭链接
cur1.close(), conn1.close()
cur2.close(), conn2.close()

# 过滤新素材
image_df = image_df[image_df.apply(lambda x:True if not any(score_df['image_id'] == x.image_id) else False, axis=1)]
image_df['score'] = np.nan
image_df['label'] = 0

In [14]:
image_df

,image_id,image_name,label_ids,game_ids,score,label
4,35916,SSR3628-横版（3D原创）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
24,35937,SSR3633-竖版（3D混剪）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
29,35942,SSR3634-竖版（3D原创）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
92,36006,SSR3641-横版（3D混剪）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
93,36007,SSR3640-竖版（3D原创）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
95,36009,SSR3642-竖版（3D原创）,20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0
96,36010,SSR3643-竖版H(3D混剪),20,1001731;1001730;1001729;1001728;1001726;100171...,NaN,0


## 观察帝国新素材
* 新素材

In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
from media_toutiao.script.realtime_df_dg import get_game_id

In [5]:
true_game_ids = get_game_id(1112)

In [6]:
# 88数据库 游标1
conn1 = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
            password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
cur1 = conn1.cursor()
sql_engine = 'set hive.execution.engine=tez'
sql = 'select image_id,label_ids,score from dws.dws_image_score_d where media_id=10 and dt>=\'2021-01-01\''
cur1.execute(sql_engine)
cur1.execute(sql)
score_df = as_pandas(cur1)

# 65数据库 游标2
conn2 = pymysql.connect(host='192.168.0.65', port=3306, user='readonly',
                        passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
cur2 = conn2.cursor(cursor=pymysql.cursors.DictCursor)
sql = '''
        /*手动查询*/ 
        SELECT
            a.image_id,
            a.image_name,
            a.label_ids,
            a.game_ids
        FROM
            db_ptom.ptom_image_info a
        WHERE
            a.create_time >= date(NOW() - INTERVAL 72 HOUR)  ## 最近3天
    '''
cur2.execute(sql)
image_df = pd.read_sql(sql, conn2)
image_df['label_ids'] = image_df['label_ids'].str.replace( ',', ';')  ## 避免分隔符冲突
image_df['game_ids'] = image_df['game_ids'].str.replace( ',', ';')  ## 避免分隔符冲突
# 新素材的label_ids对齐
image_df = image_df.loc[image_df.apply(lambda x:True if not pd.isnull(x['label_ids']) and not x['label_ids']=='' else False, axis=1)]
# 新素材的game_ids对齐
true_game_ids = [str(game_id['game_id']) for game_id in true_game_ids]
image_df = image_df.loc[image_df.apply(lambda x:True if not pd.isnull(x['game_ids']) and len(list(set(true_game_ids).intersection(set(x['game_ids'].split(';'))))) > 0 else False, axis=1)]

# 关闭链接
cur1.close(), conn1.close()
cur2.close(), conn2.close()

# 过滤新素材
image_df = image_df[image_df.apply(lambda x:True if not any(score_df['image_id'] == x.image_id) else False, axis=1)]
image_df['score'] = np.nan
image_df['label'] = 0

In [7]:
image_df

,image_id,image_name,label_ids,game_ids,score,label
15,35927,QMZZ1308-横版QP(帝国原生)（TT）,117;156,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
26,35939,QMZZ1309-横版LL（TT）,120,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
28,35941,GZDGJ595=J-LHJ+ZLM 东北口音建造,129;132;133,1001464;1001469;1001488;1001489;1001491;100149...,NaN,0
87,36001,QMZZ1317-竖版CHL（3D混剪-W）（TT）,117,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
88,36002,QMZZ1318-竖版CHL（3D混剪）（TT）,115,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
89,36003,QMZZ1319-竖版ZDC（3D复用3渲2）（TT）,115,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
90,36004,QMZZ1320-横版ZDC（3D复用3渲2）（TT）,115,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
91,36005,QMZZ1321-竖版ZDC（3D复用）(TT),124,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
94,36008,QMZZ1322-横版LL（3D混剪）(TT),116;124,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0
97,36011,QMZZ1323-横版ly（帝国原生）（TT）,156,1001714;1001713;1001702;1001700;1001693;100169...,NaN,0


## 计划报表差异化
* 非今天创建的在跑计划 vs 非今天创建的已死计划

In [5]:
from realtimedata import *

In [3]:
# 获取素材报表数据
# etl_image()

In [75]:
# 获取计划报表数据
def get_plan_detail():
    game_id = get_game_id(1056)
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
    select A.*,
    IFNULL(B.two_day_create_retain,0) two_day_create_retain,
    IFNULL(F.new_role_money,0) new_role_money,
    (case when IFNULL(A.amount,0)=0 then 0 else IFNULL(IFNULL(F.new_role_money,0),0)*1.0/IFNULL(A.amount,0) end) as new_role_rate,
    IFNULL(F.pay_role_user_num,0) pay_role_user_num,
    IFNULL(IFNULL(A.amount,0)/IFNULL(F.pay_role_user_num,0),0) as new_pay_cost
    ,(case when IFNULL(A.create_role_num,0)=0 then 0 else IFNULL(IFNULL(F.pay_role_user_num,0),0)*1.0/IFNULL(A.create_role_num,0) end) as pay_role_user_rate
  from (
     SELECT
     pp.plan_id, pp.platform,
     pp.plan_name as plan_name, pp.launch_time, pp.create_time,
     pp.media_id,
     pp.learning_type,
     pp.learning_time_dt,
     pp.learning_time_hr,
     c.*
     from (
       SELECT a.game_id as product_id,a.channel_id chl_user_id,a.source_id,
        IFNULL(sum(a.amount),0) as amount,
        IFNULL(sum(create_role_num),0) as create_role_num,
        (case when IFNULL(sum(a.create_role_num),0)=0 then IFNULL(sum(a.amount),0) else IFNULL(sum(a.amount),0)*1.0/IFNULL(sum(a.create_role_num),0) end) as create_role_cost,
        a.tdate
       from db_stdata.st_lauch_report a where a.tdate>=date(NOW() - INTERVAL 720 HOUR) and a.tdate<=date(NOW() - INTERVAL 24 HOUR) and a.tdate_type='day' and a.amount >  0 and a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
       group by a.game_id,a.channel_id,a.source_id) c

            inner join db_data_ptom.ptom_plan pp on (c.product_id=pp.game_id and c.chl_user_id=pp.chl_user_id and c.source_id=pp.source_id)


    AND pp.media_id=10  ## ================================
     ) A LEFT JOIN
       (
       select game_id,channel_id,source_id,IFNULL(sum(m.new_role_money),0) as new_role_money,
    IFNULL(sum(m.pay_role_user_num),0) as pay_role_user_num
    from (
      select a.game_id,a.channel_id,a.source_id,IFNULL(sum(a.new_role_money),0) as new_role_money,
      IFNULL(sum(a.pay_role_user_num),0) as pay_role_user_num
      from db_stdata.st_lauch_report a
      inner join db_data_ptom.ptom_plan pp on (a.game_id=pp.game_id and a.channel_id=pp.chl_user_id and a.source_id=pp.source_id)
    
      where a.tdate >=date(NOW() - INTERVAL 720 HOUR) AND a.tdate<=date(NOW() - INTERVAL 24 HOUR) AND a.tdate_type='day'
   
    AND pp.media_id=10    ## AND pp.plan_id in (732259)==================================
      group by a.game_id,a.channel_id,a.source_id 
      having (new_role_money>0 or pay_role_user_num>0)
      union ALL
      SELECT c.game_id,c.channel_id,c.source_id,sum(c.create_role_money) new_role_money,
      IFNULL(sum(c.pay_role_user_num),0) as pay_role_user_num
      from db_stdata.st_game_days c where c.report_days = 0 and c.tdate >= date(NOW() - INTERVAL 720 HOUR) and c.tdate<=date(NOW() - INTERVAL 24 HOUR) and c.tdate_type='day' and c.query_type=13
      group by c.game_id,c.channel_id,c.source_id
      having (new_role_money>0 or pay_role_user_num>0)
    ) m
    group by game_id,channel_id,source_id
    ) F ON  A.product_id=F.game_id AND A.chl_user_id=F.channel_id AND A.source_id=F.source_id
     LEFT JOIN
      (
      select game_id,channel_id,source_id,
      sum(bb.two_day_create_retain) as two_day_create_retain
        from (
          select game_id,channel_id,source_id,
        (case when retain_date=2 then create_retain_rate end) as two_day_create_retain
          from (
            select m.game_id,m.channel_id,m.source_id,m.retain_date,
            (case when IFNULL(sum(m.create_role_num),0)=0 then 0 else IFNULL(sum(m.create_role_retain_num),0)/sum(m.create_role_num) end) as create_retain_rate
            from db_stdata.st_game_retain m
            INNER JOIN db_data_ptom.ptom_plan pp on m.game_id=pp.game_id and m.channel_id=pp.chl_user_id and m.source_id=pp.source_id
            where m.tdate between date(NOW() - INTERVAL 720 HOUR) and date(NOW() - INTERVAL 24 HOUR)  and m.tdate_type='day' and m.query_type=19 and m.server_id=-1 and m.retain_date in (2)
            AND pp.media_id=10    ## ================================
            group by m.game_id,m.channel_id,m.source_id,m.retain_date
          ) cc
        )bb
        group by bb.game_id,bb.channel_id,bb.source_id
      ) B ON A.product_id=B.game_id AND A.chl_user_id=B.channel_id AND A.source_id=B.source_id
    '''
    finalSql = sql
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    
    result_df.columns = ["plan_id","platform","plan_name","model_run_datetime","create_time",
    "media_id","learning_type","learning_time_dt","learning_time_hr","game_id","channel_id","source_id","source_run_date_amount",
    "create_role_num", "create_role_cost","tdate", "create_role_retain_2d","create_role_pay_sum","create_role_pay_rate",
    "create_role_pay_num","create_role_pay_cost","create_role_roi"]  ## "data_win", deep_bid_type"出价方式
    result_df["deep_bid_type"] = result_df.apply(lambda x: random.choice(['BID_PER_ACTION', 'BID_PER_ROI']), axis=1)
    
    return result_df

In [65]:
df = get_plan_detail()
df.head()

,plan_id,platform,plan_name,model_run_datetime,create_time,media_id,learning_type,learning_time_dt,learning_time_hr,game_id,channel_id,source_id,source_run_date_amount,create_role_num,create_role_cost,tdate,create_role_retain_2d,create_role_pay_sum,create_role_pay_rate,create_role_pay_num,create_role_pay_cost,create_role_roi,deep_bid_type
0,720840,1,7tt_上午10点上上签_0628_561192814,2021-06-28,2021-06-28 10:02:14,10,None,None,None,1001379,6770,730635,401.03,5.0,80.206000,2021-07-05,0.0000,0.0,0.000000,0.0,0.0,0.00000,BID_PER_ACTION
1,720868,1,7tt_上午10点上上签_0628_561192918,2021-06-28,2021-06-28 10:02:34,10,None,None,None,1001379,6770,730663,679.64,19.0,35.770526,2021-07-05,0.1500,0.0,0.000000,0.0,0.0,0.00000,BID_PER_ROI
2,734235,1,7tt_M_闪电抓大R_0703_76101911,2021-07-03,2021-07-03 10:01:08,10,None,None,None,1001379,6770,744031,1158.01,13.0,89.077692,2021-07-05,0.0000,0.0,0.000000,0.0,0.0,0.00000,BID_PER_ROI
3,737313,1,7tt_M_闪电抓大R_0704_401031511,2021-07-04,2021-07-04 10:01:10,10,None,None,None,1001379,6770,747108,339.53,7.0,48.504286,2021-07-05,0.0000,0.0,0.000000,0.0,0.0,0.00000,BID_PER_ROI
4,737330,1,7tt_M_闪电抓大R_0704_40103091,2021-07-04,2021-07-04 10:01:32,10,None,None,None,1001379,6770,747122,1883.30,6.0,313.883333,2021-07-05,0.1667,226.0,0.120002,1.0,1883.3,0.16667,BID_PER_ACTION


In [66]:
df = df[["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","tdate","create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate",
    "learning_type","learning_time_dt","learning_time_hr"]]
df.head()

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,tdate,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,learning_type,learning_time_dt,learning_time_hr
0,6770,730635,7tt_上午10点上上签_0628_561192814,2021-06-28,2021-06-28 10:02:14,10,1001379,1,401.03,5.0,80.206000,2021-07-05,0.0,0.0,0.0,0.00000,0.000000,None,None,None
1,6770,730663,7tt_上午10点上上签_0628_561192918,2021-06-28,2021-06-28 10:02:34,10,1001379,1,679.64,19.0,35.770526,2021-07-05,0.0,0.0,0.0,0.00000,0.000000,None,None,None
2,6770,744031,7tt_M_闪电抓大R_0703_76101911,2021-07-03,2021-07-03 10:01:08,10,1001379,1,1158.01,13.0,89.077692,2021-07-05,0.0,0.0,0.0,0.00000,0.000000,None,None,None
3,6770,747108,7tt_M_闪电抓大R_0704_401031511,2021-07-04,2021-07-04 10:01:10,10,1001379,1,339.53,7.0,48.504286,2021-07-05,0.0,0.0,0.0,0.00000,0.000000,None,None,None
4,6770,747122,7tt_M_闪电抓大R_0704_40103091,2021-07-04,2021-07-04 10:01:32,10,1001379,1,1883.30,6.0,313.883333,2021-07-05,1.0,1883.3,226.0,0.16667,0.120002,None,None,None


### 中间结果

In [23]:
def get_plan_middle():
    game_id = get_game_id(1056)
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
     select A.*,
        IFNULL(B.two_day_create_retain,0) two_day_create_retain,
        IFNULL(F.new_role_money,0) new_role_money,
        (case when IFNULL(A.amount,0)=0 then 0 else IFNULL(IFNULL(F.new_role_money,0),0)*1.0/IFNULL(A.amount,0) end) as new_role_rate,
        IFNULL(F.pay_role_user_num,0) pay_role_user_num,
        IFNULL(IFNULL(A.amount,0)/IFNULL(F.pay_role_user_num,0),0) as new_pay_cost
        ,(case when IFNULL(A.create_role_num,0)=0 then 0 else IFNULL(IFNULL(F.pay_role_user_num,0),0)*1.0/IFNULL(A.create_role_num,0) end) as pay_role_user_rate
      from (
         SELECT
         pp.plan_id, pp.platform,
         pp.plan_name as plan_name, pp.launch_time,
         pp.media_id,
         pp.learning_type,
         pp.learning_time_dt,
         pp.learning_time_hr,
         c.*
         from (
           SELECT a.game_id as product_id,a.channel_id chl_user_id,a.source_id,
            IFNULL(sum(a.amount),0) as amount,
            IFNULL(sum(create_role_num),0) as create_role_num,
            (case when IFNULL(sum(a.create_role_num),0)=0 then IFNULL(sum(a.amount),0) else IFNULL(sum(a.amount),0)*1.0/IFNULL(sum(a.create_role_num),0) end) as create_role_cost
           from db_stdata.st_lauch_report a where a.tdate>={} and a.tdate<={} and a.tdate_type='day' and a.amount >  0
           group by a.game_id,a.channel_id,a.source_id) c
            
                inner join db_data_ptom.ptom_plan pp on (c.product_id=pp.game_id and c.chl_user_id=pp.chl_user_id and c.source_id=pp.source_id)
        
        
        AND pp.plan_id in (483576) AND pp.media_id=10
        
         group by pp.plan_id
     ) A LEFT JOIN
       (
       select game_id,channel_id,source_id,IFNULL(sum(m.new_role_money),0) as new_role_money,
    IFNULL(sum(m.pay_role_user_num),0) as pay_role_user_num
    from (
      select a.game_id,a.channel_id,a.source_id,IFNULL(sum(a.new_role_money),0) as new_role_money,
      IFNULL(sum(a.pay_role_user_num),0) as pay_role_user_num
      from db_stdata.st_lauch_report a
      inner join db_data_ptom.ptom_plan pp on (a.game_id=pp.game_id and a.channel_id=pp.chl_user_id and a.source_id=pp.source_id)
    
      where a.tdate ={} AND a.tdate_type='day'
   
    AND pp.plan_id in (483576) AND pp.media_id=10
      group by a.game_id,a.channel_id,a.source_id
      having (new_role_money>0 or pay_role_user_num>0)
      union ALL
      SELECT c.game_id,c.channel_id,c.source_id,sum(c.create_role_money) new_role_money,
      IFNULL(sum(c.pay_role_user_num),0) as pay_role_user_num
      from db_stdata.st_game_days c where c.report_days = 0 and c.tdate = '2021-02-18' and c.tdate_type='day' and c.query_type=13
      group by c.game_id,c.channel_id,c.source_id
      having (new_role_money>0 or pay_role_user_num>0)
    ) m
    group by game_id,channel_id,source_id
    ) F ON  A.product_id=F.game_id AND A.chl_user_id=F.channel_id AND A.source_id=F.source_id
     LEFT JOIN
      (
      select game_id,channel_id,source_id,
      sum(bb.two_day_create_retain) as two_day_create_retain
        from (
          select game_id,channel_id,source_id,
        (case when retain_date=2 then create_retain_rate end) as two_day_create_retain
          from (
            select m.game_id,m.channel_id,m.source_id,m.retain_date,
            (case when IFNULL(sum(m.create_role_num),0)=0 then 0 else IFNULL(sum(m.create_role_retain_num),0)/sum(m.create_role_num) end) as create_retain_rate
            from db_stdata.st_game_retain m
            INNER JOIN db_data_ptom.ptom_plan pp on m.game_id=pp.game_id and m.channel_id=pp.chl_user_id and m.source_id=pp.source_id
            where m.tdate between {}  and {}  and m.tdate_type='day' and m.query_type=19 and m.server_id=-1 and m.retain_date in (2)
            AND pp.plan_id in (483576) AND pp.media_id=10
            group by m.game_id,m.channel_id,m.source_id,m.retain_date
          ) cc
        )bb
        group by bb.game_id,bb.channel_id,bb.source_id
      ) B ON A.product_id=B.game_id AND A.chl_user_id=B.channel_id AND A.source_id=B.source_id
    '''  
    finalSql = sql.format('2021-02-19','2021-02-19','2021-02-19','2021-02-19','2021-02-19')
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    
    return result_df

In [24]:
df = get_plan_middle()
df.head()

,plan_id,platform,plan_name,launch_time,media_id,learning_type,learning_time_dt,learning_time_hr,product_id,chl_user_id,source_id,amount,create_role_num,create_role_cost,two_day_create_retain,new_role_money,new_role_rate,pay_role_user_num,new_pay_cost,pay_role_user_rate


## 规则交叉验证

In [76]:
def plan_ptom_rule():
    """ State_1的计划启停规则 """
    
    '''
    表所有字段
    ["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","tdate", create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate","create_role_pay_num_cum",
    "learning_type","learning_time_dt","learning_time_hr","deep_bid_type"]
    '''
    
    data = get_plan_detail()
    data = data[["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","tdate","create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate",
    "learning_type","learning_time_dt","learning_time_hr","deep_bid_type"]]

    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据
    # 去重，对当天运行进行监控情况
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[data['source_run_date_amount'] >= 300]

    # 增加每次付费跑法的判断
    data_pre_action = data_not_ignore[data_not_ignore['deep_bid_type'] == 'BID_PER_ACTION']
    data_not_ignore = data_not_ignore[data_not_ignore['deep_bid_type'] != 'BID_PER_ACTION']

    data_win_0 = data_not_ignore[data_not_ignore['platform'] == 1]
    data_win_0_ios = data_not_ignore[data_not_ignore['platform'] == 2]
    
    data_win_0_pre = data_pre_action[data_pre_action['platform'] == 1]
    data_win_0_ios_pre = data_pre_action[data_pre_action['platform'] == 2]
    
    result_df = pd.DataFrame()

    # 1:关; 0:开；
    # win=0预判
    # 每次付费安卓
    if data_win_0_pre.shape[0] != 0:
        # data_win_0_pre['label'] = data_win_0_pre.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
        #                                                               (x.create_role_cost >= 350) else (
        #     1 if (x.create_role_pay_sum <= 24) & (x.create_role_cost >= 400) else
        #     (1 if (x.create_role_roi <= 0.018) & (x.create_role_cost >= 450) else (
        #         1 if x.create_role_cost >= 500 else 0))), axis=1)

        data_win_0_pre['y_pred'] = data_win_0_pre.apply(
            lambda x: 1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount <= 1000) &
                           (x.create_role_cost >= 150) else (
                1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount > 1000) &
                     (x.create_role_cost >= 100) else (
                    1 if (x.create_role_roi > 0) & (x.create_role_roi <= 0.02) & (x.create_role_cost >= 150) else (
                        1 if x.create_role_cost >= 200 else 0))), axis=1)

        result_df = result_df.append(data_win_0_pre)

    # 每次付费ISO
    if data_win_0_ios_pre.shape[0] != 0:
        # data_win_0_ios_pre['label'] = data_win_0_ios_pre.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
        #                                                                       (x.create_role_cost >= 400) else (
        #     1 if (x.create_role_pay_sum <= 24) & (x.create_role_cost >= 450) else
        #     (1 if (x.create_role_roi <= 0.014) & (x.create_role_cost >= 500) else (
        #         1 if x.create_role_cost >= 550 else 0))), axis=1)
        data_win_0_ios_pre['y_pred'] = data_win_0_ios_pre.apply(
            lambda x: 1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount <= 1000) &
                           (x.create_role_cost >= 250) else (
                1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount > 1000) &
                     (x.create_role_cost >= 200) else (
                    1 if (x.create_role_roi <= 0.02) & (x.create_role_cost >= 300) else (
                        1 if x.create_role_cost >= 300 else 0))), axis=1)
        result_df = result_df.append(data_win_0_ios_pre)

    # 其它付费方式安卓
    if data_win_0.shape[0] != 0:
        # data_win_0['label'] = data_win_0.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
        #                                                       (x.create_role_cost >= 160) else (
        #     1 if (x.create_role_pay_sum <= 24) & (x.create_role_cost >= 200) else
        #     (1 if (x.create_role_roi <= 0.018) & (x.create_role_cost >= 240) else (
        #         1 if x.create_role_cost >= 300 else 0))), axis=1)
        data_win_0['y_pred'] = data_win_0.apply(
            lambda x: 1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount <= 1000) &
                           (x.create_role_cost >= 120) else (
                1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount > 1000) &
                     (x.create_role_cost >= 70) else (
                    1 if (x.create_role_roi > 0) & (x.create_role_roi <= 0.02) & (x.create_role_cost >= 100) else (
                        1 if x.create_role_cost >= 130 else 0))), axis=1)
        result_df = result_df.append(data_win_0)

    # 其它付费方式IOS
    if data_win_0_ios.shape[0] != 0:
        # data_win_0_ios['label'] = data_win_0_ios.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
        #                                                               (x.create_role_cost >= 180) else (
        #     1 if (x.create_role_pay_sum < 24) & (x.create_role_cost >= 200) else
        #     (1 if (x.create_role_roi <= 0.014) & (x.create_role_cost >= 250) else (
        #         1 if x.create_role_cost >= 300 else 0))), axis=1)
        data_win_0_ios['y_pred'] = data_win_0_ios.apply(
            lambda x: 1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount <= 1000) &
                           (x.create_role_cost >= 200) else (
                1 if (x.create_role_pay_sum == 0) & (x.source_run_date_amount > 1000) &
                     (x.create_role_cost >= 150) else (
                    1 if (x.create_role_roi <= 0.02) & (x.create_role_cost >= 250) else (
                        1 if x.create_role_cost >= 250 else 0))), axis=1)
        result_df = result_df.append(data_win_0_ios)
    # 凌晨前关闭消耗大于1500且ROI<1.8%的计划
    d_time = datetime.datetime.strptime(str(datetime.datetime.now().date()) + '23:40', '%Y-%m-%d%H:%M')
    d_time1 = datetime.datetime.strptime(str((datetime.datetime.now() + datetime.timedelta(days=1)).date()) + '00:01',
                                         '%Y-%m-%d%H:%M')
    n_time = datetime.datetime.now()
    if n_time > d_time and n_time < d_time1:
        result_df['y_pred'] = result_df.apply(
            lambda x: 1 if (x.source_run_date_amount >= 800) & (x.create_role_roi <= 0.018) else x.label, axis=1)
    
    return result_df

In [77]:
# def budget_ptom_rule():
result_df = plan_ptom_rule()
result_df

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,tdate,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,learning_type,learning_time_dt,learning_time_hr,deep_bid_type,y_pred
0,6770,668011,7tt_自动建计划2.0__0528_161023115,2021-05-28,2021-05-28 14:01:33,10,1001379,1,68204.48,1253.0,54.432945,2021-06-08,16.0,4262.78,904.0,0.01277,0.013254,None,None,None,BID_PER_ACTION,0
1,6770,677897,7tt_自建大哥大__0603_62081324,2021-06-03,2021-06-03 10:12:32,10,1001379,1,52316.22,830.0,63.031590,2021-06-08,11.0,4756.02,1184.0,0.01325,0.022632,None,None,None,BID_PER_ACTION,0
2,6770,680947,7tt_自建大哥大__0604_20098180,2021-06-04,2021-06-04 10:01:32,10,1001379,1,2803.74,29.0,96.680690,2021-06-08,1.0,2803.74,12.0,0.03448,0.004280,None,None,None,BID_PER_ACTION,0
3,6770,681504,7tt_今天也要打败机器人__0604_64099585,2021-06-04,2021-06-04 14:01:32,10,1001379,1,12264.15,226.0,54.266150,2021-06-08,3.0,4088.05,286.0,0.01327,0.023320,None,None,None,BID_PER_ACTION,0
4,6770,681852,7tt_自建大哥大__0604_15078830,2021-06-04,2021-06-04 15:26:32,10,1001379,1,867.48,6.0,144.580000,2021-06-08,2.0,433.74,42.0,0.33333,0.048416,None,None,None,BID_PER_ACTION,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15868,21339,757516,0707挑战6策略70_无_3595_43,2021-07-07,2021-07-07 17:36:31,10,1001731,1,371.41,0.0,371.410000,2021-07-07,0.0,0.00,0.0,0.00000,0.000000,None,None,None,BID_PER_ROI,1
15872,21339,757523,0707挑战6策略70_年龄过滤APP_3609_5,2021-07-07,2021-07-07 17:36:31,10,1001731,1,473.46,0.0,473.460000,2021-07-07,0.0,0.00,0.0,0.00000,0.000000,None,None,None,BID_PER_ROI,1
15880,21339,757531,0707挑战6策略70_男智投_2138_23,2021-07-07,2021-07-07 17:38:12,10,1001731,1,394.13,0.0,394.130000,2021-07-07,0.0,0.00,0.0,0.00000,0.000000,None,None,None,BID_PER_ROI,1
15883,21339,757534,0707挑战6策略70_无_2363_37,2021-07-07,2021-07-07 17:40:11,10,1001731,1,450.11,0.0,450.110000,2021-07-07,0.0,0.00,0.0,0.00000,0.000000,None,None,None,BID_PER_ROI,1


In [78]:
def cross_valid():
    """ 超参数寻优：交叉验证 """
    # todo: y_true也就是每条计划统计当日实际是关闭还是打开的，怎么确定
    

In [12]:
('1'*10 + '0'*14 + '1'*6 + '0'*6 + '1' * 12) * 4 + '0'*48 + ('0'*16 + '1'*32) + ('1'*12 + '0'*7 + '1'*29)

'111111111100000000000000111111000000111111111111111111111100000000000000111111000000111111111111111111111100000000000000111111000000111111111111111111111100000000000000111111000000111111111111000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111000000011111111111111111111111111111'